In [2]:
## use RNA-STAR conda environment
from pathlib import Path
import traceback
import argparse
import pandas as pd
import numpy as np
import re
from scipy.stats import fisher_exact

In [ ]:
def match_regex(folder_name):
    """
    Given input folder names, extract the group name.
        EXAMPLE: '7KO-Cyto-BS_processed_fastqs' -> '7KO-Cyto'
    """
    try:
        match = re.match(r"(.+)-(?:BS|NBS)_processed_fastqs", folder_name)
    except Exception as e:
        print(f"Failed to match input folder to group with RegEx: {e}")
        traceback.print_exc()
        raise
    return match.group(1) ## return first regex capture

In [ ]:
def create_columns(merged_colnames, fisher_colnames, df_merged):
   """
   1. Takes all columns from merged df
      and organizes them by BS/NBS type 
   2. Sums up corresponding bases and deletions
      and creates new columns in df
   """
   bs_base_pattern = re.compile(r".+_(A|C|G|T)_BS$")
   nbs_base_pattern = re.compile(r".+_(A|C|G|T)_NBS$")
   bs_del_pattern = re.compile(r".+_(Deletions)_BS$")
   nbs_del_pattern = re.compile(r".+_(Deletions)_NBS$")
    
   pattern_dict = {"Bases_BS": [col for col in merged_colnames if bs_base_pattern.match(col)],
                   "Bases_NBS": [col for col in merged_colnames if nbs_base_pattern.match(col)],
                   "Deletions_BS": [col for col in merged_colnames if bs_del_pattern.match(col)],
                   "Deletions_NBS": [col for col in merged_colnames if nbs_del_pattern.match(col)]}

   try:
      for newcol, key in zip(fisher_colnames, pattern_dict):
         df_merged[newcol] = df_merged[pattern_dict[key]].sum(axis=1)
         
   except Exception as e:
      print(f"Failed to create new columns for Fisher's Exact Test: {e}")
      traceback.print_exc()
      raise

In [ ]:
class FilterTSV:
    def merged_output(df_merged):
        """
        1. Creates 4 new columns
        2. Selects new columns in df
           * Reshapes each row into 2x2 matrix
           * Runs Fisher's Exact Test
           * Appends p-value column
        """
        merged_colnames = df_merged.columns.tolist()
        fisher_colnames = ["TotalBases_BS", "TotalBases_NBS", "TotalDeletions_BS", "TotalDeletions_NBS"]
        create_columns(merged_colnames, fisher_colnames, df_merged) ## add 4 new columns
        df_merged["p_value"] = df_merged[fisher_colnames].apply(lambda row: fisher_exact(row.values.reshape(2, 2))[1], axis=1) ## each row is reshaped into 2x2 matrix
        df_merged = df_merged.drop(columns=["index"])
    
    def filtered_output():
        text

    def discarded_output():
        text

In [ ]:
## main code
def clean_output(folder_name):
    """
    Filters .tsv files in grouped folders
    """
    current_path = Path.cwd()
    group_name = match_regex(folder_name)
    input_folder = current_path/"calculations"/group_name/"individual_tsv"
    processed_folder = current_path/"calculations"/group_name

    try: 
        if input_folder.is_dir():
            tsv_list = [*input_folder.glob("*.tsv")] ## collect paths of tsv files and put in a list            
            num = ["df%s" %s for s in range(1,7)] ## creates a list of strings: df1, df2, ..., df6
            listcomp = [pd.read_csv(i, sep = "\t") for i in tsv_list] ## reads in all tsv files as pandas df; access 1st df w/ listcomp[0], etc.
            df_dict = dict(zip(num, listcomp))

            ## Merge pandas dataframes
            colnames = df_dict["df1"].columns.tolist()
            selected_colnames = ["index"] + colnames[0:17] ## columns that are always the same throughout all dfs

            for i in num:
                if i=0:
                    ## merge df1 + df2
                    df_merged = pd.merge(df_dict[i].reset_index(), df_dict[i+1].reset_index(), on = selected_colnames, how = "outer")
                elif i=5:
                    break
                else:
                    ## for remaining dfs after df1 + df2, overwrite existing var: df_merged = df_merged + df_dict[i+1]
                    df_merged = pd.merge(df_merged, df_dict[i+1].reset_index(), on = selected_colnames, how = "outer")

            ## Initialize class
            filtertsv = FilterTSV()

            ## Save merged .tsv (all_sites)
            filtertsv.merged_output(df_merged)
            df_merged.to_csv(f"{processed_folder}/{group_name}_all_sites.tsv", sep = "\t", index = False)

            ## Save null .tsv (no_data)
            null_rows = df_merged.isnull().any(axis=1)
            df_null = df_merged[null_rows].copy()
            df_null.to_csv(f"{processed_folder}/{group_name}_no_data.tsv", sep = "\t", index = False)

            ## Save filtered .tsv (priority_filtered)

            ## Save discarded .tsv (non_sites)

    except Exception as e:
        print(f"Failed to create merged .tsv file: {e}")
        traceback.print_exc()
        raise

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description = "Filters .tsv outputs from calculate_dr script.")
    parser.add_argument("--folder_name", help = "Name of processed_fastqs folder", required = True)
    args = parser.parse_args()

    print("Filtering .tsv files...")
    clean_output(args.folder_name)
    print("Process finished.")

### Extra code

In [ ]:
# Conditional mean
df = pd.read_csv("data.csv")

col = "my_column"
threshold = 10

filtered = df[df[col] > threshold]

if filtered[col].mean() > threshold:
    df = filtered
else:
    # If you want, keep original df or do something else
    pass

###################

## Fisher's Exact Tests
from scipy.stats import fisher_exact

# df with columns: 'A', 'B', 'C', 'D'
def get_pvalue(row):
    table = [[row['A'], row['B']], [row['C'], row['D']]]
    _, p = fisher_exact(table)
    return p

df['p_value'] = df.apply(get_pvalue, axis=1)